## Aggregating
A dataset containing daily sales can be aggregated to show monthly sales instead. This section introduces _grouping_ and _pivoting_, two common operations for aggregating data.

In [1]:
import pandas as pd

In [2]:
baby = pd.read_csv("chapter6/babynames.txt")
baby

,Name,Sex,Count,Year
0,Liam,M,19659,2020
1,Noah,M,18252,2020
2,Oliver,M,14147,2020
3,Elijah,M,13034,2020
4,William,M,12541,2020
...,...,...,...,...
2020717,Ula,F,5,1880
2020718,Vannie,F,5,1880
2020719,Verona,F,5,1880
2020720,Vertie,F,5,1880


Let's say we want to find out the total number of babies born as recorded in this data.

In [3]:
baby['Count'].sum()

np.int64(352554503)

Let's say we instead want to answer a more interesting question: are US births trending upward over time?

We call this operation _grouping_ followed by _aggregating_.

In [6]:
baby.groupby('Year')['Count'].sum().reset_index()
# reset_index() makes the 'Year' column to be a data column from an index column, then we can use the data to draw the trend line

,Year,Count
0,1880,194419
1,1881,185772
2,1882,213385
3,1883,208932
4,1884,234244
...,...,...
136,2016,3642755
137,2017,3548457
138,2018,3487193
139,2019,3437438


In [7]:
counts_by_year = baby.groupby('Year')['Count'].sum().reset_index()

In [8]:
import plotly.express as px

In [9]:
px.line(counts_by_year, x='Year', y='Count', width=350, height=250)

We can also group by multiple columns by passing a list of column names to groupby().

In [10]:
baby.groupby(['Year', 'Sex'])['Count'].sum()

Year  Sex
1880  F        83929
      M       110490
1881  F        85034
      M       100738
1882  F        99699
              ...   
2018  M      1810309
2019  F      1651911
      M      1785527
2020  F      1581301
      M      1706423
Name: Count, Length: 282, dtype: int64

In [12]:
counts_by_year_sex = baby.groupby(['Year', 'Sex'])['Count'].sum().reset_index()
fig = px.line(
    counts_by_year_sex,
    x='Year', y='Count', color='Sex', # color parameter creates separate lines for M and F
    title='Baby Counts by Year and Sex',
    width=350,
    height=250
)
fig.show()

#### Example: Using .value_counts()
One of the more common dataframe tasks is to count the number of times every unique item in a column appears.

In [17]:
names = ['Eden', 'Sachit', 'Eden', 'Sachit', 'Sachit', 'Luke']
classroom = pd.DataFrame(names, columns=['name'])
classroom

,name
0,Eden
1,Sachit
2,Eden
3,Sachit
4,Sachit
5,Luke


In [18]:
# Find out the number of times each name appears in the dataframe above
classroom.groupby('name')['name'].size()

name
Eden      2
Luke      1
Sachit    3
Name: name, dtype: int64

```pandas``` provides a shorthand ```.value_counts()``` method for pd.Series object:

In [19]:
classroom['name'].value_counts()

name
Sachit    3
Eden      2
Luke      1
Name: count, dtype: int64

*By default, the ```.value_counts()``` method will sort the resulting series from **highest to lowest number**, making it convenient to see the most and least common values.

#### Custom Aggregation Functions
```.mean()```, ```.size()```, ```.first()```, ```.max()```

In [20]:
baby.groupby('Year')['Count'].max()

Year
1880     9655
1881     8769
1882     9557
1883     8894
1884     9388
        ...  
2016    19380
2017    18824
2018    19924
2019    20555
2020    19659
Name: Count, Length: 141, dtype: int64

We can define and use a custom aggregation function. ```pandas``` lets us do this through ```.agg(fn)```, where ```fn``` is a function that we define.

For instance, if we want to find the difference between the largest and smallest values within each group (the range of data), we could first define a function called ```data_range```, then pass that function into ```.agg()```:

In [21]:
def data_range(counts):
    return counts.max() - counts.min()

baby.groupby('Year')['Count'].agg(data_range)

Year
1880     9650
1881     8764
1882     9552
1883     8889
1884     9383
        ...  
2016    19375
2017    18819
2018    19919
2019    20550
2020    19654
Name: Count, Length: 141, dtype: int64

We start to defining a ```count_unique``` function that counts the number of unique values in series. Then we pass that function into ```.agg()```:

In [22]:
def count_unique(s):
    return len(s.unique())

unique_names_by_year = (
    baby.groupby('Year')['Name'].agg(count_unique)
)
unique_names_by_year

Year
1880     1889
1881     1829
1882     2012
1883     1962
1884     2158
        ...  
2016    30431
2017    30042
2018    29619
2019    29417
2020    28613
Name: Name, Length: 141, dtype: int64

In [24]:
px.line(unique_names_by_year.reset_index(),
        x='Year', y='Name', labels={'Name': '#unique names'},
        width=350, height=250)

We see that the number of unique names has generally increased over time, even though the number of babies born annually has plateaued since the 1960s.

#### Pivoting
Earlier in this section we grouped the baby names data by year and sex:

In [25]:
counts_by_year_sex = (baby
                      .groupby(['Year', 'Sex'])
                      ['Count'].sum()
)
counts_by_year_sex

Year  Sex
1880  F        83929
      M       110490
1881  F        85034
      M       100738
1882  F        99699
              ...   
2018  M      1810309
2019  F      1651911
      M      1785527
2020  F      1581301
      M      1706423
Name: Count, Length: 282, dtype: int64

We can imagine the same data with the ```Sex``` index level "pivoted" to the columns of a dataframe.

In [27]:
mf_pivot = pd.pivot_table(
    baby,
    index='Year',    # Column to turn into new index
    columns='Sex', # Column to turn into new columns
    values='Count', # Column to aggregate for values
    aggfunc='sum'    # Aggregation function
)
mf_pivot

Sex,F,M
Year,,
1880,83929,110490
1881,85034,100738
1882,99699,113686
1883,104307,104625
1884,119802,114442
...,...,...
2016,1748838,1893917
2017,1702303,1846154
2018,1676884,1810309


Pivot tables are useful for quickly summarizing data using two attributes and are often seen in articles and papers.

In [28]:
fig = px.line(mf_pivot, width=350, height=250)
fig.update_traces(selector=1, line_dash='dashdot')
fig.update_yaxes(title='Value')